In [1]:
#uses Registered2to1Adder_NIR - v1.1

In [2]:
#inReadyC default to simplified definition

In [3]:
import sys
import math

In [4]:
#N>=1
N = input('Enter N: ')
N = int(N)

Enter N: 30


In [5]:
#HRR>=1
HRR = input('Enter HRR (empty or 1 for noHR) : ')

Enter HRR (empty or 1 for noHR) : 2


In [6]:
if N<1:
    N=1
if not HRR:
    HRR=1
else:
    HRR = int(HRR)
    if HRR<1:
        HRR=1
    if HRR>N:
        HRR=N
if HRR>1:
    NOIPSMin = math.ceil(N/HRR);
    NOIPSMax = math.ceil(N/(HRR-1))-1;
    NOIPSMM = (NOIPSMin!=NOIPSMax)

In [7]:
#>=NOIPSMin, <=NOIPSMax
if HRR>1 and NOIPSMM:
    NOIPS = input('Enter Number of inputs per series (>={}, <={}, empty for {}) : '.format(NOIPSMin, NOIPSMax, NOIPSMin))

Enter Number of inputs per series (>=15, <=29, empty for 15) : 20


In [8]:
#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH default value (empty for 10) : ')

Enter IN_WIDTH default value (empty for 10) : 


In [9]:
_Enable_Input_Registers = input('Enable Input Registers? (empty or 0 to disable, 1 to enable): ')

Enable Input Registers? (empty or 0 to disable, 1 to enable): 


In [10]:
if HRR>1:
    if NOIPSMM:
        if not NOIPS:
            NAdd=NOIPSMin
        else:
            NOIPS = int(NOIPS)
            if NOIPS<NOIPSMin:
                NAdd=NOIPSMin
            elif NOIPS>NOIPSMax:
                NAdd=NOIPSMax
            else:
                NAdd=NOIPS
    else:
        NAdd=NOIPSMin
else:
    NAdd=N
if not IN_WIDTH:
    IN_WIDTH=10
else:
    IN_WIDTH = int(IN_WIDTH)
    if IN_WIDTH<1:
        IN_WIDTH=1
if not _Enable_Input_Registers:
    _Enable_Input_Registers=0
else:
    _Enable_Input_Registers = int(_Enable_Input_Registers)

In [11]:
__Print_To_File = 1

if __Print_To_File<=0:
    of=sys.stdout
else:
    if(HRR==1):
        if _Enable_Input_Registers>0:
            of=open("./VectorAdd_{}_noHR_IR.v".format(N), 'w+')
        else:
            of=open("./VectorAdd_{}_noHR_NIR.v".format(N), 'w+')
    else:
        if _Enable_Input_Registers>0:
            of=open("./VectorAdd_{}_HR_x{}_p{}_series_IR.v".format(N, HRR, NAdd), 'w+')
        else:
            of=open("./VectorAdd_{}_HR_x{}_p{}_series_NIR.v".format(N, HRR, NAdd), 'w+')        

In [12]:
print("`timescale 1ns / 1ps\n", file=of)
if HRR==1:
    if _Enable_Input_Registers>0:
        print("module VectorAdd_{}_noHR_IR".format(N), file=of)
    else:
        print("module VectorAdd_{}_noHR_NIR".format(N), file=of)
else:
    if _Enable_Input_Registers>0:
        print("module VectorAdd_{}_HR_x{}_p{}_series_IR".format(N, HRR, NAdd), file=of)
    else:
        print("module VectorAdd_{}_HR_x{}_p{}_series_NIR".format(N, HRR, NAdd), file=of)        
print("#(", file=of)
print("parameter IN_WIDTH = {}".format(IN_WIDTH), file=of)
print(")(", file=of)
print("input clk, reset, enable,", file=of)
if HRR>1:
    print("output reg readyForNewDataSeries = 1,", file=of)
lghrr=math.ceil(math.log2(HRR))
if lghrr==1:
    print("output reg inSeries = 0,", file=of)
elif lghrr>1:
    print("output reg [{}:0] inSeries = 0,".format(lghrr-1), file=of)
print("input inReady,", file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(NAdd):
    print("A{}, ".format(i), end='', file=of)
print(file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(NAdd):
    print("B{}, ".format(i), end='', file=of)
print(file=of)
if HRR==1:
    print("output outReady,", file=of)
    NE=NAdd
else:
    NE = N - NAdd*(HRR-1)
    while NE<=0:
        NE += NAdd
    if NE==1:
        print("output S0outReady,", file=of)
    else:
        print("output S0toS{}outReady,".format(NE-1), file=of)
    if NE==NAdd:
        print("output reg SNoutReady=0, //not used", file=of)
    elif NE==NAdd-1:
        print("output S{}outReady,".format(NE), file=of)
    else:
        print("output S{}toS{}outReady,".format(NE, NAdd-1), file=of)
if lghrr==1:
    print("output reg outSeries = 1,", file=of)
elif lghrr>1:
    print("output reg [{}:0] outSeries = {},".format(lghrr-1, HRR-1), file=of)
print("output signed [IN_WIDTH:0] ", end='', file=of)
for i in range(NAdd):
    print("S{}, ".format(i), end='', file=of)
print(file=of)
if HRR==1:
    print("output earlyOutReady", file=of)
else:
    if NE==1:
        print("output S0earlyOutReady,", file=of)
    else:
        print("output S0toS{}earlyOutReady,".format(NE-1), file=of)
    if NE==NAdd:
        print("output reg SNearlyOutReady=0 //not used", file=of)
    elif NE==NAdd-1:
        print("output S{}earlyOutReady".format(NE), file=of)
    else:
        print("output S{}toS{}earlyOutReady".format(NE, NAdd-1), file=of)
print(");\n", file=of)

In [13]:
if HRR!=1:
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\treadyForNewDataSeries <= 1;", file=of)
    print("\t\tinSeries <= 0;", file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    print("\t\tif(inReady) begin", file=of)
    print("\t\t\tif(inSeries=={}) begin".format(HRR-1), file=of)
    print("\t\t\t\treadyForNewDataSeries <= 1;", file=of)
    print("\t\t\t\tinSeries <= 0;", file=of)
    print("\t\t\tend", file=of)
    print("\t\t\telse begin", file=of)
    print("\t\t\t\treadyForNewDataSeries <= 0;", file=of)
    if HRR==2:
        print("\t\t\t\tinSeries <= 1;", file=of)    
    else:
        print("\t\t\t\tinSeries <= inSeries+1;", file=of)
    print("\t\t\tend", file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end", file=of)
    print(file=of)

In [14]:
if _Enable_Input_Registers<=0:
    if NE!=NAdd: #HRR!=1 and 
        print("wire inReadyC = (inSeries!={}) & inReady;".format(HRR-1), file=of)
        print(file=of)
else:
    print("reg inReadyR;", file=of)
    if NE!=NAdd:
        print("reg inReadyCR;", file=of)
    print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(NAdd-1):
        print("A{}R, ".format(i), end='', file=of)
    print("A{}R;".format(NAdd-1), file=of)
    print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(NAdd-1):
        print("B{}R, ".format(i), end='', file=of)
    print("B{}R;".format(NAdd-1), file=of)
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\tinReadyR <= 0;", file=of)
    if NE!=NAdd:
        print("\t\tinReadyCR <= 0;", file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    print("\t\tinReadyR <= inReady;", file=of)
    if NE!=NAdd:
        print("\t\tinReadyCR <= (inSeries!={}) & inReady;".format(HRR-1), file=of)
    print("\t\tif(inReady) begin", file=of)
    for i in range(NE):
        print("\t\t\tA{}R <= A{}; B{}R <= B{};".format(i, i, i, i), file=of)
    if NE!=NAdd:
        print("\t\t\tif(inSeries!={}) begin".format(HRR-1), file=of)
        for i in range(NE, NAdd):
            print("\t\t\t\tA{}R <= A{}; B{}R <= B{};".format(i, i, i, i), file=of)
        print("\t\t\tend", file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end", file=of)
    print(file=of)

In [15]:
print("""//(* use_dsp48 = "yes" *) //yes, no, AutoMax, Auto""", file=of)

In [16]:
print("Registered2to1Adder_NIR #(.IN_WIDTH(IN_WIDTH))\nAdd0 (clk, reset, enable,", file=of)
if _Enable_Input_Registers<=0:
    print("inReady,", file=of)
    print("A0, B0,", file=of)
else:
    print("inReadyR,", file=of)
    print("A0R, B0R,", file=of)
if HRR==1:
    print("OutReady,", file=of)
else:
    if NE==1:
        print("S0outReady,", file=of)
    else:
        print("S0toS{}outReady,".format(NE-1), file=of)
print("S0,", file=of)
if HRR==1:
    print("earlyOutReady);\n", file=of)
else:
    if NE==1:
        print("S0earlyOutReady);\n", file=of)
    else:
        print("S0toS{}earlyOutReady);\n".format(NE-1), file=of)
for i in range(1, NE):
    print("Registered2to1Adder_NIR #(.IN_WIDTH(IN_WIDTH))\nAdd{} (clk, reset, enable,".format(i), file=of)
    if _Enable_Input_Registers<=0:
        print("inReady,", file=of)
        print("A{}, B{},".format(i, i), file=of)
    else:
        print("inReadyR,", file=of)
        print("A{}R, B{}R,".format(i, i), file=of)
    print("aor{}, //not used\nS{},\naeor{}); //not used\n".format(i, i, i), file=of)
if NE!=NAdd:
    print("Registered2to1Adder_NIR #(.IN_WIDTH(IN_WIDTH))\nAdd{} (clk, reset, enable,".format(NE), file=of)
    if _Enable_Input_Registers<=0:
        print("inReadyC,", file=of)
        print("A{}, B{},".format(NE, NE), file=of)
    else:
        print("inReadyCR,", file=of)
        print("A{}R, B{}R,".format(NE, NE), file=of)
    if NE==NAdd-1:
        print("S{}outReady,".format(NE), file=of)
    else:
        print("S{}toS{}outReady,".format(NE, NAdd-1), file=of)
    print("S{},".format(NE), file=of)
    if NE==NAdd-1:
        print("S{}earlyOutReady);\n".format(NE), file=of)
    else:
        print("S{}toS{}earlyOutReady);\n".format(NE, NAdd-1), file=of)    
for i in range(NE+1, NAdd):
    print("Registered2to1Adder_NIR #(.IN_WIDTH(IN_WIDTH))\nAdd{} (clk, reset, enable,".format(i, i, i, i, i, i), file=of)
    if _Enable_Input_Registers<=0:
        print("inReadyC,", file=of)
        print("A{}, B{},".format(i, i), file=of)
    else:
        print("inReadyCR,", file=of)
        print("A{}R, B{}R,".format(i, i), file=of)
    print("aor{}, //not used\nS{},\naeor{}); //not used\n".format(i, i, i), file=of)

In [17]:
if HRR!=1:
    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    print("\t\toutSeries <= {};".format(HRR-1), file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    if NE==1:
        print("\t\tif(S0earlyOutReady) begin", file=of)
    else:
        print("\t\tif(S0toS{}earlyOutReady) begin".format(NE-1), file=of)
    print("\t\t\tif(outSeries=={}) begin".format(HRR-1), file=of)
    print("\t\t\t\toutSeries <= 0;", file=of)
    print("\t\t\tend", file=of)
    print("\t\t\telse begin", file=of)
    if HRR==2:
        print("\t\t\t\toutSeries <= 1;", file=of)
    else:
        print("\t\t\t\toutSeries <= outSeries+1;", file=of)
    print("\t\t\tend", file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end", file=of)
    print(file=of)

In [18]:
print("endmodule", file=of)

In [19]:
if __Print_To_File>0:
    of.close()